In [75]:
torch.cuda.empty_cache()

In [2]:
import os
SAVE_PATH = 'generations/'

try:
    os.mkdir(SAVE_PATH)
except FileExistsError:
    pass

## Text2Image

In [1]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler

model_id = 'stable-diffusion-v1-4'
lms = LMSDiscreteScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear"
)
safety_checker = lambda images, **kwargs: (images, False)

device = 'cuda:0'
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, scheduler=lms, use_auth_token=True)
pipe.safety_checker = safety_checker
pipe = pipe.to(device)

/raid/alisher_amantay/.conda/envs/stab-diff/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
n_images = 1

prompt = "photo of football soccer player hitting the ball with immense force, hyperrealistic, full shot, action"
height = 512
width = 512

# run pipeline in inference (sample random noise and denoise)
with autocast('cuda'):
    images = pipe([prompt]*n_images, num_inference_steps=50,
                  height=height, width=width,
                  eta=0.3, guidance_scale=9.0)["sample"]


100%|██████████| 50/50 [00:04<00:00, 10.73it/s]


In [ ]:
images[0]

In [64]:
# save a single image
idx = 0
images[idx].save(SAVE_PATH + f'{prompt[:200]}_{idx}.png')

In [65]:
# save all images
for idx, image in enumerate(images):
    image.save(SAVE_PATH + f'{prompt}_{idx}.png')

## Image2Image

In [40]:
from PIL import Image
import torch
from torch import autocast
from image_to_image import StableDiffusionImg2ImgPipeline, preprocess

safety_checker = lambda images, **kwargs: (images, False)
model_id = 'stable-diffusion-v1-4'
device = 'cuda:1'

i2i_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True
)
i2i_pipe.safety_checker = safety_checker
i2i_pipe = i2i_pipe.to(device)

In [23]:
# load img from system
init_path = 'img_path.png'
init_image = Image.open(init_path).convert("RGB")
init_image = init_image.resize((832, 832))
init_image = preprocess(init_image)

In [71]:
# load img from memory
init_image = images[0].convert('RGB')
init_image = init_image.resize((832, 512))
init_image = preprocess(init_image)

In [ ]:
# run generation
n_images = 1
prompt = "cheering poster for a figure skating show, dark silhouette of a female figure-skater, beautiful, gradients, smooth, extreme details"

with autocast("cuda"):
    i2i_images = i2i_pipe(prompt=[prompt] * n_images, init_image=init_image, strength=0.7, guidance_scale=7.5)["sample"]

i2i_images[0]

In [73]:
# load output as input
init_image = i2i_images[0].convert('RGB')
init_image = init_image.resize((832, 512))
init_image = preprocess(init_image)

In [ ]:
# run generation
n_images = 1
prompt = "cheering poster for a figure skating show, dark silhouette of a female figure-skater with skates on, beautiful, gradients, smooth, extreme details"

with autocast("cuda"):
    i2i_images = i2i_pipe(prompt=[prompt] * n_images, init_image=init_image, strength=0.6, guidance_scale=7.5, num_inference_steps=50)["sample"]

i2i_images[0]

In [76]:
# save an image
idx = 5
i2i_images[0].save(SAVE_PATH + f'{prompt[:200]}_{idx}.png')

# Interpolate between prompts

In [2]:
from stable_diffusion_videos.stable_diffusion_pipeline import StableDiffusionPipeline as VidPipe
from stable_diffusion_videos.stable_diffusion_pipeline import NoCheck
from stable_diffusion_videos import walk

model_id = 'stable-diffusion-v1-4'

device = 'cuda:8'
pipe = VidPipe.from_pretrained(
    model_id, use_auth_token=True, safety_checker=NoCheck()
).to(device)

You have passed a non-standard module NoCheck(). We cannot verify whether it has the correct type
{'trained_betas'} was not found in config. Values will be initialized to default values.


In [22]:
walk(
    pipeline=pipe,
    prompts=['oil painting of a king penguin looking to the left left left in snowy tundra, side view, full shot, hyperrealistic, trending on artstation', 'oil painting of a deer looking to the left in the forest, side view, full shot, hyperrealistic, trending on artstation'],
    seeds=[42, 42],
    output_dir='generations',     # Where images/videos will be saved
    name='penguin_deer2',     # Subdirectory of output_dir where images/videos will be saved
    guidance_scale=8.5,      # Higher adheres to prompt more, lower lets model take the wheel
    num_steps=200,             # Change to 60-200 for better results...3-5 for testing
    num_inference_steps=50, 
    scheduler='klms',        # One of: "klms", "default", "ddim"
    disable_tqdm=True,      # Set to True to disable tqdm progress bar
    make_video=True,         # If false, just save images
    use_lerp_for_text=True,  # Use lerp for text embeddings instead of slerp
    do_loop=False,           # Change to True if you want last prompt to loop back to first prompt
)

COUNT: 1/200
COUNT: 20/200
COUNT: 40/200
COUNT: 60/200
COUNT: 80/200
COUNT: 100/200
COUNT: 120/200
COUNT: 140/200
COUNT: 160/200
COUNT: 180/200
COUNT: 200/200


ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

'generations/penguin_deer2/penguin_deer2.mp4'